In [ ]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-11-25
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-11-25
@Title : Ideation Project - 5. Fetching and transforming output stream files from Kinesis Data Stream and upload them to S3 Bucket to be read by Redshift.

'''

In [ ]:
import boto3
import json
import pandas as pd

In [ ]:
import os
os.environ['AWS_DEFAULT_REGION'] = 'ap-south-1'
os.environ['AWS_ACCESS_KEY_ID'] = os.environ.get('test1_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.environ.get('test1_secret_access_key')

In [ ]:
def connect_to_s3_client():
    """

    Description:
        To connect to AWS S3 service.
    Parameter:
        No parameters
    Return:
        ServiceResource s3
    """
    # s3 =  boto3.resource('s3')
    client = boto3.client("s3")
    return client


def connect_to_s3_resource():
    """

    Description:
        To connect to AWS S3 service through an IAM user.
    Parameter:
        No parameters
    Return:
        ServiceResource s3
    """
    s3 =  boto3.resource(service_name = 's3')
    return s3


def file_to_csv(filename):
    """

    Description:
        To read records/dictionaries from text file and convert it into a .csv file.
    Parameter:
        string filename
    Return:
        Dataframe df
    """
    with open(f"streaming_output/{filename}.txt" , encoding="utf8") as file:
            data = file.read()
            list_of_incomp_str_dict = data.split('}')
    
    # print(list_of_incomp_str_dict)

    list_of_comp_str_dict = []

    for inc_dict in list_of_incomp_str_dict:
        list_of_comp_str_dict.append(inc_dict+"}")

    list_of_comp_str_dict.pop()
    # print(list_of_comp_str_dict)

    #Now we convert dictionary as string to dictionary
    list_of_dict = []

    for str_dict in list_of_comp_str_dict:
        list_of_dict.append(json.loads(str_dict))

    df = pd.DataFrame(list_of_dict)
    df.to_csv(f'streaming_output\{filename}.csv', index=False, header=True)
    
    return df

In [ ]:
client = connect_to_s3_client()

filename = 'dataset_output_stream'
client.download_file('output-stream-bucket', 'data/2022/11/27/21/dataset-output-downstream-1-2022-11-27-21-27-56-865201e7-ded1-4432-89a5-1346a975acd8',f'streaming_output/{filename}.txt')

file_to_csv(filename)

# creating new bucket 
s3 = connect_to_s3_resource()
client.create_bucket(Bucket = 'redshift-dataset-input',ACL = 'public-read-write', CreateBucketConfiguration={'LocationConstraint': 'ap-south-1'})

# uploading csv file to the new bucket to be accessed by redshift 
client.upload_file(Filename = f'streaming_output\{filename}.csv',Bucket = 'redshift-dataset-input', Key = f'{filename}.csv')

In [ ]:
filename = 'title_output_stream'
client.download_file('output-stream-bucket', 'data/2022/11/27/21/dataset-output-downstream-1-2022-11-27-21-38-06-46b677af-8235-4e1a-b033-3a021bd8f072',f'streaming_output/{filename}.txt')

file_to_csv(filename)

# uploading csv file to the new bucket to be accessed by redshift 
client.upload_file(Filename = f'streaming_output\{filename}.csv',Bucket = 'redshift-dataset-input', Key = f'{filename}.csv')

In [ ]:
filename = 'body_output_stream'
client.download_file('output-stream-bucket', 'data/2022/11/27/22/dataset-output-downstream-1-2022-11-27-22-08-59-f488159f-f493-4743-9c78-0b53db63cbcb',f'streaming_output/{filename}.txt')

file_to_csv(filename)

# uploading csv file to the new bucket to be accessed by redshift 
client.upload_file(Filename = f'streaming_output\{filename}.csv',Bucket = 'redshift-dataset-input', Key = f'{filename}.csv')